In [9]:
import numpy as np
import cv2
import math

font = cv2.FONT_HERSHEY_COMPLEX

def createROI(frame, x=100, y=400):
    roi=frame[x:y, x:y]
    cv2.rectangle(frame,(x,x),(y,y), (255,0,0), 0) 
    return roi

def extractHueSat(frame):
    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h, s, _ = cv2.split(hsvImage)
    return h,s

def segmentImageBySkinTone(frame):
    hue, sat = extractHueSat(frame)
    hue = np.divide(hue, 255.0)
    sat = np.divide(sat, 255.0)
    mask = hue[:]

    skinToneMatch = np.logical_and(np.where(hue <= 0.15, True, False), np.where(sat < 0.5, True, False))
    mask[skinToneMatch] = 1

    saturationOfBackground = np.where((sat > 0.75) & (sat < 0.13), True, False)
    hueOfBackground = np.where(hue <= 0.90, True, False)
    mask[saturationOfBackground] = 0
    mask[hueOfBackground] = 0

    mask = (mask*255).astype(np.uint8)
    mask = cv2.dilate(mask, np.ones((3,3)), iterations=1)
    mask = cv2.GaussianBlur(mask, (5,5), 100)
    return mask

def getTriangleDetails(start, end, far):
    # find length of all sides of triangle
    triangleSides = list()
    triangleSides.append(math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2))
    triangleSides.append(math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2))
    triangleSides.append(math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2))
    semiPerimeter = sum(triangleSides)/2
    triangleArea = math.sqrt(semiPerimeter * (semiPerimeter - triangleSides[0]) * (semiPerimeter - triangleSides[1]) * (semiPerimeter - triangleSides[2]))

    #distance between point and convex hull
    dist = (2*triangleArea)/triangleSides[0]

    # apply cosine rule here
    angle = math.acos((triangleSides[1]**2 + triangleSides[2]**2 - triangleSides[0]**2) / (2 * triangleSides[1] * triangleSides[2])) * 57
    
    return dist, angle

def findGesture(frame, numDefects, areaOfContour):
    if numDefects == 1:
        if areaOfContour < 2000:
            cv2.putText(frame,'Place hand in the box',(0,50), font, 1.5, (0,0,255), 3, cv2.LINE_AA)
        else:
            cv2.putText(frame,'I choose Paper!',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

    elif numDefects == 2:
        cv2.putText(frame,'I choose Rock!',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

    elif numDefects == 5:
        cv2.putText(frame,'I choose Scissors!',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

    else:
        cv2.putText(frame,'Can you try again please?',(10,50), font, 1.5, (0,0,255), 3, cv2.LINE_AA)
        
def hysteresis(edges):
    size = np.shape(edges)
    if size >= 0:
        return
    relatedComponents = np.zeros(size)
    numComponents = 0
    for i in range(size[0]):
        for j in range(size[1]):
            if(edges[i,j] >= 2 and relatedComponents[i,j] == 0):
                numComponents += 1
                listOfConnectedPoints = []
                listOfConnectedPoints.append((i,j))
                
                while(listOfConnectedPoints != []):
                    p = listOfConnectedPoints.pop()
                    relatedComponents[p[0],p[1]] = numComponents
                    if(p[0] < size[0]-1 and edges[p[0]+1,p[1]] >= 1 and relatedComponents[p[0]+1,p[1]] != numComponents):
                        listOfConnectedPoints.append((p[0]+1,p[1]))
                    if(p[1] < size[1]-1 and edges[p[0],p[1]+1] >= 1 and relatedComponents[p[0],p[1]+1] != numComponents):
                        listOfConnectedPoints.append((p[0],p[1]+1))
                    if(p[1] < size[1]-1 and p[0] < size[0]-1 and edges[p[0]+1,p[1]+1] >= 1 and relatedComponents[p[0]+1,p[1]+1] != numComponents):
                        listOfConnectedPoints.append((p[0]+1,p[1]+1))
                    if(p[0] > 0 and edges[p[0]-1,p[1]] >= 1 and relatedComponents[p[0]-1,p[1]] != numComponents):
                        listOfConnectedPoints.append((p[0]-1,p[1]))
                    if(p[1] > 0 and edges[p[0],p[1]-1] >= 1 and relatedComponents[p[0],p[1]-1] != numComponents):
                        listOfConnectedPoints.append((p[0],p[1]-1))
                    if(p[0] > 0 and p[1] > 0 and edges[p[0]-1,p[1]-1] >= 1 and relatedComponents[p[0]-1,p[1]-1] != numComponents):
                        listOfConnectedPoints.append((p[0]-1,p[1]-1))
                    if(p[0] > 0 and p[1] < size[1]-1 and edges[p[0]-1,p[1]+1] >= 1 and relatedComponents[p[0]-1,p[1]+1] != numComponents):
                        listOfConnectedPoints.append((p[0]-1,p[1]+1))
                    if(p[0] < size[0]-1 and p[1] > 0 and edges[p[0]+1,p[1]-1] >= 1 and relatedComponents[p[0]+1,p[1]-1] != numComponents):
                        listOfConnectedPoints.append((p[0]+1,p[1]-1))
    return(relatedComponents)


def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0]) #To find the points that belong to the convex hull by determing the points most "clockwise" to each other
        
def generateConvexHull(roi, contour, hull):
    contour1 = contour.tolist() #The contour co-ordinates in np.array format to python lists

    contour1 = sorted(contour1)

    lower = []
    for i in range(len(contour1)):
        p = contour1[i][0]
        while (len(lower)>=2) and (cross(lower[-2], lower[-1], p) <= 0):
            lower.pop()
        lower.append(p)       #determines the points in the lower half of the convex hull

    upper = []
    if len(contour1) > 0:
        for i in range(len(contour1)-1,-1,-1):
            p = contour1[i][0]
            while (len(upper)>=2) and (cross(upper[-2], upper[-1], p) <= 0):
                upper.pop()
            upper.append(p)     #determines the points in the upper half

    points = lower[:-1] + upper[:-1]      #The total set of points
    

    points = np.array(points)          
    mask = np.zeros((roi.shape[0], roi.shape[1]))
    cv2.fillConvexPoly(mask, points, 1)
    mask = mask.astype(np.bool)
    return mask

#     out = np.zeros_like(roi)        #displaying only the contour hull part of the image
#     out[mask] = roi[mask]
#     cv2.imshow('Extracted Image', out)


def play():
    cap = cv2.VideoCapture(0)
    while True:
        flag, frame = cap.read()
        if not flag:
            break
                
        frame = cv2.flip(frame,1)
        
        try:
            # Define a Region of Interest.
            roi = createROI(frame, 100, 400)
            
            # Segment Image By the skin colour.
            binaryImage = segmentImageBySkinTone(roi)
            
            # Find Contours and Optimize them.
            contours = cv2.findContours(binaryImage, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)[1]
            contuors = hysteresis(contours)
            
            # If no contours found, loop back and keep trying to capture the video
            if len(contours) == 0:
                continue

            # Finding the hand By Assuming it has the largest Contour Area
            handContour = max(contours, key = lambda cont: cv2.contourArea(cont))
            areaOfContour = cv2.contourArea(handContour)

            # Creating a hull around the contours in order to understand the shape that the hand forms.
            hull = cv2.convexHull(handContour, returnPoints=False)
            generateConvexHull(binaryImage, handContour, hull)

            # Finding the convex cavities/defects inside the hull to distinguish between the fingers.
            defectsFound = cv2.convexityDefects(handContour, hull)
            numDefects = 0

            # Find the number of defects, i.e., the number of fingers held up by the user.
            for defect in range(defectsFound.shape[0]):
                points = defectsFound[defect,0]
                vertex1 = tuple(handContour[points[0]][0])
                vertex2 = tuple(handContour[points[1]][0])
                vertex3 = tuple(handContour[points[2]][0])

                dist, angle = getTriangleDetails(vertex1, vertex2, vertex3)

                # Only consider defects where the angles are greater than 75deg and the distance
                # between the apex point of the hull and the defect is actually considerable and not just
                # a result of noise. Calculated by trial and error.
                if angle <= 75 and dist > 26:
                    numDefects += 1
                    cv2.circle(roi, vertex3, 3, [0,0,255], -1)

                # Draw the generated hull around the hand
                cv2.line(roi, vertex1, vertex2, [0,0,255], 2)


            numDefects+=1

            # Find the gesture from the resulting number of defects.
            findGesture(frame, numDefects, areaOfContour)

            cv2.imshow('Rock-Paper-Scissors!', frame)
            
            if cv2.waitKey(5) == ord('q'):
                cv2.destroyAllWindows()
                cap.release()

    
        except Exception as e:
            print e
            pass

play()